https://www.kesci.com/home/project/5b6acf1d9889570010c88af1?from=groupmessage

In [1]:
import tensorflow as tf
import numpy as np
import pprint
import re

/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#对字符串进行分词

text = '''然而 ， 即使 这样 ， 大数据 （ 分析 ） 对于 绝 大 部分 人 来 说 仍 停留 在 概念 层面 ， 或者 是 给人 很 “ 高 大 上 ” 、 不接地气 的 印象 ，
因为 这个 词 似乎 和 “ 技术 ” 、 “ 编程 ” 紧密 联系 在 一起 。

其实 不然 ， 经过 这几年 的 发展 ， 大数据 应用 已经 触手可及 ，
比如 笔者 上面 用到 的 谷歌趋势 ，
以及 互联网 从业者 很 熟悉 的 百度指数、 微指数 等  ， 都是 一些 大家 触手可及 的 大数据分析 工具 。
借助 这些 工具 ， 我们 只 需要 在 了解 业务知识 的 情况 下 ，就 能 完成 一些 大数据趋势分析 、 事件 （ 传播 ） 分析 、用户画像 分析 等 。

在 本文 中 ， 笔者 将会 介绍 大数据 分析 主要 的 处理 对象 — 社会化媒体 （ Social Media ） ，
以及 对 社会化媒体 进行分析 的 重要手段 — — 社会化聆听 （ Social  Listening ） 。 最后 ， 笔者 将 以 汽车 行业 的 一个 实操 案例 ， 来 讲述 如何 使用 社会化聆听 （ 工具 ）来 对 社会化媒体 大数据 进行 分析 ， 虽然 “ 隔行如隔山 ” ， 但 “ 隔行不隔理 ”， 其他 领域 的 读者 也 可以 借鉴 这种 分析思路 和 方法 ， 来 帮助 自己 在 产品设计 / 运营 、 市场调研 中 达成 目标 。'''

tokenized = text.split(' ')
tokenized = [i.strip().strip('|、|，|。(|)|\n')  for i in tokenized]
tokenized = [i.strip()  for i in tokenized if len(i) >0]
vocab = {k:v for v,k in enumerate(np.unique(tokenized))}
vocab

{'/': 0,
 'Listening': 1,
 'Media': 2,
 'Social': 3,
 '—': 4,
 '“': 5,
 '”': 6,
 '一个': 7,
 '一些': 8,
 '一起': 9,
 '上': 10,
 '上面': 11,
 '下': 12,
 '不接地气': 13,
 '不然': 14,
 '业务知识': 15,
 '中': 16,
 '主要': 17,
 '也': 18,
 '了解': 19,
 '事件': 20,
 '互联网': 21,
 '产品设计': 22,
 '人': 23,
 '介绍': 24,
 '仍': 25,
 '从业者': 26,
 '以': 27,
 '以及': 28,
 '传播': 29,
 '似乎': 30,
 '但': 31,
 '使用': 32,
 '借助': 33,
 '借鉴': 34,
 '停留': 35,
 '其他': 36,
 '其实': 37,
 '分析': 38,
 '分析思路': 39,
 '印象': 40,
 '即使': 41,
 '发展': 42,
 '只': 43,
 '可以': 44,
 '和': 45,
 '因为': 46,
 '在': 47,
 '处理': 48,
 '大': 49,
 '大家': 50,
 '大数据': 51,
 '大数据分析': 52,
 '大数据趋势分析': 53,
 '如何': 54,
 '完成': 55,
 '实操': 56,
 '对': 57,
 '对于': 58,
 '对象': 59,
 '将': 60,
 '将会': 61,
 '就': 62,
 '层面': 63,
 '工具': 64,
 '已经': 65,
 '市场调研': 66,
 '帮助': 67,
 '应用': 68,
 '很': 69,
 '微指数': 70,
 '情况': 71,
 '我们': 72,
 '或者': 73,
 '技术': 74,
 '方法': 75,
 '是': 76,
 '最后': 77,
 '本文': 78,
 '来': 79,
 '案例': 80,
 '概念': 81,
 '比如': 82,
 '汽车': 83,
 '然而': 84,
 '熟悉': 85,
 '用到': 86,
 '用户画像': 87,
 '百度指数': 88,
 '的': 89,
 '目

现在，我们需要设定词向量的大小（Embedding size），也就是每个词汇向量的维度（the dimension of each vector），在这里，我将维数设定为50，词汇长度则是词汇表的词汇数量（len(vocab)）



In [3]:
EMBED_SIZE = 50
VOCAB_LEN = len(vocab.keys())

print(VOCAB_LEN)

131


我们知道需要设定我们想要嵌入的词汇（words）的id。 下面，我们将采用“新浪微热点”和“新浪微舆情”这两个词汇作为示例：


In [4]:
words_ids = tf.constant([vocab["百度指数"], vocab["大数据"]])

记住，words_ids代表词汇表（vocabolary）中某些词汇的ID。 词汇表（vocabolary）是从词汇（tokens）到ID的映射。

WHY 为什么我们需要这么做? 神经网络只能输入数字, 所以我们需要将数字传给嵌入层（embedding layer）

In [9]:
embeddings = tf.Variable(tf.random_uniform([VOCAB_LEN, EMBED_SIZE]))
embed = tf.nn.embedding_lookup(embeddings, words_ids)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(embed))

[[0.74391127 0.02930129 0.38832843 0.3631438  0.43186665 0.45800328
  0.9129416  0.2808609  0.9585413  0.8009238  0.35639906 0.49088562
  0.12949634 0.5944823  0.6212542  0.8009771  0.5463362  0.24998045
  0.9633652  0.3514458  0.13889301 0.5647832  0.48341334 0.6642556
  0.68493557 0.9660964  0.31583393 0.51296854 0.13751388 0.8680496
  0.5509461  0.2295897  0.03370833 0.496992   0.31215584 0.05183256
  0.78546715 0.981969   0.14507663 0.7488055  0.6125692  0.32402527
  0.3705089  0.56136465 0.41353774 0.8478087  0.37777352 0.72365654
  0.80633986 0.5472679 ]
 [0.21012688 0.21627796 0.18510914 0.7879895  0.10957861 0.5343449
  0.3254664  0.13342118 0.83059764 0.14732063 0.6508204  0.95676494
  0.6126257  0.955376   0.7836976  0.7706263  0.55114603 0.29065287
  0.41188657 0.37956345 0.82569003 0.58541584 0.7692239  0.6933913
  0.48301625 0.29444826 0.30220413 0.3588251  0.86243665 0.17175126
  0.17906988 0.8980262  0.9128767  0.51094115 0.6202601  0.2792374
  0.73681223 0.9730966  0.15